In [2]:
import pandas as pd

from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

# Group an ID by consecutive dates
Calculate the number of consecutive days for a given ID. If there is a gap of days for an ID, we should capture both streaks as different rows

In [3]:
df1 = pd.DataFrame({'ID': [1, 1, 1, 1, 2, 2, 2, 2],
                    'Date': ['2017-01-07', '2017-01-08', '2017-01-09', '2017-01-23',
                             '2017-01-05', '2017-01-06', '2017-01-10', '2017-01-11']
                    })
df1['Date'] = pd.to_datetime(df1['Date'])
df1

,ID,Date
0,1,2017-01-07
1,1,2017-01-08
2,1,2017-01-09
3,1,2017-01-23
4,2,2017-01-05
5,2,2017-01-06
6,2,2017-01-10
7,2,2017-01-11


#### PYTHON - Method 1: using diff for datetime datatype

In [4]:
# In order for SHIFT to work properly, we would probably need to sort the dataframe

# 1. Is there more than 1 day difference with the previous day? (use the not equal method ne(1))
df1['is_there_more_than_one_day_difference'] = df1.groupby('ID')['Date'].diff().dt.days.ne(1)

# 2. Group the booleans by using cumsum()
df1['streak_id'] = df1['is_there_more_than_one_day_difference'].cumsum()

# Calculate the size of each grouped_streaks by ID
df1['streak_size_days'] = df1.groupby(['ID', 'streak_id'])['streak_id'].transform('size')

# With this we could extract, for each ID, what is the longest streak
df1['longest_streak_rank'] = df1.groupby('ID')['streak_size_days'].rank(method='dense', ascending=False)
df1

,ID,Date,is_there_more_than_one_day_difference,streak_id,streak_size_days,longest_streak_rank
0,1,2017-01-07,True,1,3,1.0
1,1,2017-01-08,False,1,3,1.0
2,1,2017-01-09,False,1,3,1.0
3,1,2017-01-23,True,2,1,2.0
4,2,2017-01-05,True,3,2,1.0
5,2,2017-01-06,False,3,2,1.0
6,2,2017-01-10,True,4,2,1.0
7,2,2017-01-11,False,4,2,1.0


In [5]:
df1[['ID', 'streak_size_days', 'longest_streak_rank']].drop_duplicates().sort_values(['ID','longest_streak_rank'])

,ID,streak_size_days,longest_streak_rank
0,1,3,1.0
3,1,1,2.0
4,2,2,1.0


#### SQL

In [6]:
df1 = pd.DataFrame({'ID': [1, 1, 1, 1, 2, 2, 2, 2],
                    'Date': ['2017-01-07', '2017-01-08', '2017-01-09', '2017-01-23',
                             '2017-01-05', '2017-01-06', '2017-01-10', '2017-01-11']
                    })
df1['Date'] = pd.to_datetime(df1['Date'])

In [7]:
# In order for LAG to work properly, we would probably need to sort the dataframe
query = "WITH previous_date_df AS (" \
        "SELECT ID, " \
        "       Date AS first_date, " \
        "       COALESCE(LAG(Date) OVER (PARTITION BY ID ORDER BY Date), Date) AS previous_date " \
        "FROM df1), " \
        "date_difference_is_not_one_df AS (" \
        "SELECT *, " \
        "       CASE WHEN (julianday(first_date) - (julianday(previous_date))) != 1 THEN True ELSE False END AS is_there_more_than_one_day_difference " \
        "FROM previous_date_df), " \
        "grouped_streaks_df AS (" \
        "SELECT *, " \
        "       SUM(is_there_more_than_one_day_difference) OVER (ORDER BY ID, first_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as streak_id " \
        "FROM date_difference_is_not_one_df) " \
        "SELECT ID, streak_id, COUNT(*) AS streak_size_days FROM grouped_streaks_df GROUP BY ID, streak_id "

mysql(query)

,ID,streak_id,streak_size_days
0,1,1,3
1,1,2,1
2,2,3,2
3,2,4,2


# Groupby an ID by consecutive events
For example, wins and losses

In [8]:
df2 = pd.DataFrame({'Group':['A','A', 'A','A','A','A','B','B','B','B','B','B','B'],
                   'Score':['win', 'loss', 'loss', 'loss', 'win', 'win', 'win', 'win', 'win', 'loss', 'win', 'loss', 'loss']})
df2

,Group,Score
0,A,win
1,A,loss
2,A,loss
3,A,loss
4,A,win
5,A,win
6,B,win
7,B,win
8,B,win
9,B,loss


#### PYTHON - Overall win streak

In [9]:
# 1. Extract previous score by using the shift() method
df2['previous_score'] = df2['Score'].shift(periods=1)

# 2. Compare if they are not equal
df2['is_score_not_equal_to_previous'] = df2['Score'] != df2['previous_score']

# 3. Calculate the grouped scores streaks by using cumsum() and the booleans from is_score_equal_to_previous
df2['streak_id'] = df2['is_score_not_equal_to_previous'].cumsum()

# 4. Calculate the streaks
df2['cumulative_streaks'] = df2.groupby('streak_id')['Score'].cumcount()+1

df2

,Group,Score,previous_score,is_score_not_equal_to_previous,streak_id,cumulative_streaks
0,A,win,NaN,True,1,1
1,A,loss,win,True,2,1
2,A,loss,loss,False,2,2
3,A,loss,loss,False,2,3
4,A,win,loss,True,3,1
5,A,win,win,False,3,2
6,B,win,win,False,3,3
7,B,win,win,False,3,4
8,B,win,win,False,3,5
9,B,loss,win,True,4,1


#### SQL - Overall win streak

In [10]:
df2 = pd.DataFrame({'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                    'Group_':['A','A', 'A','A','A','A','B','B','B','B','B','B','B'],
                   'Score':['win', 'loss', 'loss', 'loss', 'win', 'win', 'win', 'win', 'win', 'loss', 'win', 'loss', 'loss']})

# In order for LAG to work properly, we would probably need to sort the dataframe
query = "WITH previous_score_df AS (" \
        "SELECT ID, " \
        "       Score AS first_score, " \
        "       COALESCE(LAG(Score) OVER (ORDER BY ID, Score), Score) AS previous_score " \
        "FROM df2), " \
        "are_scores_equal_df AS (" \
        "SELECT *, " \
        "       CASE WHEN first_score != previous_score THEN True ELSE False END AS is_previous_score_equal " \
        "FROM previous_score_df), " \
        "grouped_streaks_df AS (" \
        "SELECT *, " \
        "       SUM(is_previous_score_equal) OVER (ORDER BY ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as streak_id " \
        "FROM are_scores_equal_df) " \
        "SELECT first_score, streak_id, COUNT(*) AS streak_size_days " \
        "FROM grouped_streaks_df " \
        "GROUP BY first_score, streak_id " \
        "ORDER BY streak_id"

mysql(query)

,first_score,streak_id,streak_size_days
0,win,0,1
1,loss,1,3
2,win,2,5
3,loss,3,1
4,win,4,1
5,loss,5,2


#### PYTHON - Win streak by group

In [11]:
df2 = pd.DataFrame({'Group':['A','A', 'A','A','A','A','B','B','B','B','B','B','B'],
                   'Score':['win', 'loss', 'loss', 'loss', 'win', 'win', 'win', 'win', 'win', 'loss', 'win', 'loss', 'loss']})

# 1. Extract previous score by using the shift() method
df2['previous_score'] = df2.groupby(['Group'])['Score'].shift(periods=1)

# 2. Compare if they are equal
df2['is_score_equal_to_previous'] = df2['Score'] != df2['previous_score']

# 3. Calculate the grouped scores streaks by using cumsum() and the booleans from is_score_equal_to_previous
df2['equal_grouped_scores'] = df2['is_score_equal_to_previous'].cumsum()

# 4. Calculate the streaks
df2['streaks'] = df2.groupby('equal_grouped_scores')['Score'].cumcount()+1

df2

,Group,Score,previous_score,is_score_equal_to_previous,equal_grouped_scores,streaks
0,A,win,NaN,True,1,1
1,A,loss,win,True,2,1
2,A,loss,loss,False,2,2
3,A,loss,loss,False,2,3
4,A,win,loss,True,3,1
5,A,win,win,False,3,2
6,B,win,NaN,True,4,1
7,B,win,win,False,4,2
8,B,win,win,False,4,3
9,B,loss,win,True,5,1


#### SQL - Win streak by group

In [12]:
df2 = pd.DataFrame({'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                    'Group_':['A','A', 'A','A','A','A','B','B','B','B','B','B','B'],
                   'Score':['win', 'loss', 'loss', 'loss', 'win', 'win', 'win', 'win', 'win', 'loss', 'win', 'loss', 'loss']})

# In order for LAG to work properly, we would probably need to sort the dataframe
query = "WITH previous_score_df AS (" \
        "SELECT ID, " \
        "       Group_, " \
        "       Score AS first_score, " \
        "       COALESCE(LAG(Score) OVER (PARTITION BY Group_ ORDER BY ID, Score), Score) AS previous_score " \
        "FROM df2), " \
        "are_scores_equal_df AS (" \
        "SELECT *, " \
        "       CASE WHEN first_score != previous_score THEN True ELSE False END AS is_previous_score_equal " \
        "FROM previous_score_df), " \
        "grouped_streaks_df AS (" \
        "SELECT *, " \
        "       SUM(is_previous_score_equal) OVER (ORDER BY ID, Group_ ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as streak_id " \
        "FROM are_scores_equal_df) " \
        "SELECT Group_, first_score, streak_id, COUNT(*) AS streak_size_days " \
        "FROM grouped_streaks_df " \
        "GROUP BY Group_, first_score, streak_id " \
        "ORDER BY streak_id"

mysql(query)

,Group_,first_score,streak_id,streak_size_days
0,A,win,0,1
1,A,loss,1,3
2,A,win,2,2
3,B,win,2,3
4,B,loss,3,1
5,B,win,4,1
6,B,loss,5,2
